In [270]:
%use kandy
%use lib-ext

In [271]:
@file:DependsOn("com.ustermetrics:ecos4j:2.0.1")
@file:DependsOn("com.ustermetrics:ecos4j-native:1.0.4-2.0.10")
@file:DependsOn("org.ejml:ejml-all:0.43.1")

In [272]:
import com.ustermetrics.ecos4j.Model
import com.ustermetrics.ecos4j.Parameters
import com.ustermetrics.ecos4j.Status.OPTIMAL
import org.ejml.data.DMatrixSparseCSC
import org.ejml.dense.row.factory.DecompositionFactory_DDRM
import org.ejml.ops.DConvertMatrixStruct
import org.ejml.simple.SimpleMatrix

# Portfolio Optimization
        
A long-only investor wishes to maximize the expected portfolio return given a limit on the portfolio risk

$$
\begin{align*}
& \text{maximize} & & \mu^T x \\
& \text{subject to} & & x^T \Sigma x \leq \sigma^2 \\
& & & \mathbf{1} x = 1 \\
& & & x \geq 0
\end{align*}
$$

where $x$ is the unknown vector of portfolio allocations, $\mu$ is the estimated expected return vector, $\Sigma$ is the
estimated covariance matrix, and $\sigma$ is the given limit on the portfolio risk.

In [273]:
// Define portfolio optimization problems
val mu = SimpleMatrix(doubleArrayOf(0.05, 0.09, 0.07, 0.06))
val sigma = SimpleMatrix(
    4, 4, true,
    0.0016, 0.0006, 0.0008, -0.0004,
    0.0006, 0.0225, 0.0015, -0.0015,
    0.0008, 0.0015, 0.0025, -0.001,
    -0.0004, -0.0015, -0.001, 0.01
)
val sigmaLimits = List(20) { 0.0321 + it * (0.06 - 0.0321) / 19 }

In [274]:
// Problem dimension
val n = mu.numRows

In [275]:
// Compute Cholesky decomposition of sigma
val chol = DecompositionFactory_DDRM.chol(n, true)
if (!chol.decompose(sigma.copy().getMatrix()))
    throw IllegalArgumentException("Cholesky decomposition failed")
val upTriMat = SimpleMatrix.wrap(chol.getT(null)).transpose()

// Define second-order cone program
val cMat = mu.negative()
    .concatRows(SimpleMatrix(1, 1))
println("\ncMat")
cMat.print()

val aMat = SimpleMatrix.ones(1, n)
    .concatColumns(SimpleMatrix(1, 1))
println("\naMat")
aMat.print()

val bMat = SimpleMatrix.ones(1, 1)
println("\nbMat")
bMat.print()

val gMatPosOrt = SimpleMatrix.identity(n)
    .negative()
    .concatColumns(SimpleMatrix(n, 1))
    .concatRows(SimpleMatrix(1, n).concatColumns(SimpleMatrix.ones(1, 1)))
val gMatSoc = SimpleMatrix(1, n)
    .concatColumns(SimpleMatrix.filled(1, 1, -1.0))
    .concatRows(upTriMat.negative().concatColumns(SimpleMatrix(n, 1)))
val gMat = gMatPosOrt.concatRows(gMatSoc)
println("\ngMat")
gMat.print()

val hMat = SimpleMatrix(2 * n + 2, 1)


cMat
Type = DDRM , rows = 5 , cols = 1
-.05       
-.09       
-.07       
-.06       
 0         

aMat
Type = DDRM , rows = 1 , cols = 5
 1           1           1           1           0         

bMat
Type = DDRM , rows = 1 , cols = 1
 1         

gMat
Type = DDRM , rows = 10 , cols = 5
-1          -0          -0          -0           0         
-0          -1          -0          -0           0         
-0          -0          -1          -0           0         
-0          -0          -0          -1           0         
 0           0           0           0           1         
 0           0           0           0          -1         
-.04        -.015       -.02         .01         0         
-0          -.149248116 -.008040303  .00904534   0         
-0          -0          -.045114893  .016120458  0         
-0          -0          -0          -.097766623  0         


In [276]:
// ecos4j needs sparse aMat and gMat
val tol = 1e-8
val aSpMat = DConvertMatrixStruct.convert(aMat.ddrm, null as DMatrixSparseCSC?, tol)
println("\naSpMat")
aSpMat.print()

val gSpMat = DConvertMatrixStruct.convert(gMat.ddrm, null as DMatrixSparseCSC?, tol)
println("\ngSpMat")
gSpMat.print()


aSpMat
Type = DSCC , rows = 1 , cols = 5 , nz_length = 4
 1           1           1           1               *     

gSpMat
Type = DSCC , rows = 10 , cols = 5 , nz_length = 16
-1               *           *           *           *     
     *      -1               *           *           *     
     *           *      -1               *           *     
     *           *           *      -1               *     
     *           *           *           *       1         
     *           *           *           *      -1         
-.04        -.015       -.02         .01             *     
     *      -.149248116 -.008040303  .00904534       *     
     *           *      -.045114893  .016120458      *     
     *           *           *      -.097766623      *     


In [277]:
// Helper function
fun toLongArray(arr: IntArray): LongArray {
    return arr.map { it.toLong() }.toLongArray()
}

In [278]:
val muPortfolio = mutableListOf<Double>()
val sigmaPortfolio = mutableListOf<Double>()
lateinit var xMat: SimpleMatrix

// Compute efficient frontier
sigmaLimits.forEach { sigmaLimit ->

    // Set sigma limit
    hMat.set(n, 0, sigmaLimit)

    Model().use { model ->

        // Set up model
        model.setup(
            n + 1L, longArrayOf(n + 1L), 0, gSpMat.nz_values, toLongArray(gSpMat.col_idx),
            toLongArray(gSpMat.nz_rows), cMat.ddrm.data, hMat.ddrm.data, aSpMat.nz_values,
            toLongArray(aSpMat.col_idx), toLongArray(aSpMat.nz_rows), bMat.ddrm.data
        )

        // Create and set parameters
        val parameters = Parameters.builder()
            .verbose(true)
            .build()
        model.setParameters(parameters)

        // Optimize model
        val status = model.optimize()
        if (status != OPTIMAL)
            throw IllegalStateException("Optimization failed")

        // Get solution
        xMat = SimpleMatrix(model.x())
            .extractMatrix(0, n, 0, 1)

        // Compute portfolio return and risk
        muPortfolio.add(mu.transpose().mult(xMat).get(0, 0))
        sigmaPortfolio.add(sqrt(xMat.transpose().mult(sigma).mult(xMat).get(0, 0)))
    }
}

In [279]:
// Plot efficient frontier
plot {
    layout.title = "Efficient frontier"
    smoothLine(sigmaPortfolio, muPortfolio, method = SmoothMethod.LOESS(span = 0.1)) {
        color = Color.GREEN
    }
    points {
        x(sigmaPortfolio) {
            axis {
                name = "Risk"
            }
        }
        y(muPortfolio) {
            axis {
                name = "Return"
            }
        }
        size = 3.0
        color = Color.GREEN
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wYo9NZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Efficient frontier"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Risk",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Return",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.03210000361425526,0.032381821759425805,0.03266363990459636,0.0329454580497669,0.03322727619493745,0.033509094340107994,0.033790912485278546,0.03407273063044909,0.03435454877561964,0.03463636692079018,0.034918185065960734,0.03520000321113128,0.035481821356301825,0.03576363950147238,0.03604545764664292,0.03632727579181347,0.036609093936984013,0.036890912082154566,0.03717273022732511,0.03745454837249566,0.03773636651766621,0.038018184662836754,0.0383000028080073,0.038581820953177845,0.0388636390983484,0.03914545724351894,0.03942727538868949,0.03970909353386004,0.039990911679030586,0.04027272982420113,0.040554547969371676,0.04083636611454223,0.041118184259712774,0.04140000240488332,0.04168182055005387,0.04196363869522442,0.04224545684039496,0.04252727498556551,0.04280909313073606,0.043090911275906606,0.04337272942107715,0.0436545475662477,0.04393636571141825,0.044218183856588794,0.04450000200175934,0.04478182014692989,0.04506363829210044,0.04534545643727098,0.045627274582441535,0.04590909272761208,0.046190910872782626,0.04647272901795317,0.046754547163123716,0.04703636530829427,0.047318183453464814,0.047600001598635366,0.04788181974380591,0.04816363788897646,0.048445456034147,0.04872727417931755,0.0490090923244881,0.049290910469658646,0.0495727286148292,0.04985454675999974,0.05013636490517029,0.050418183050340834,0.05070000119551138,0.05098181934068193,0.05126363748585248,0.05154545563102303,0.051827273776193575,0.05210909192136412,0.052390910066534666,0.05267272821170521,0.05295454635687576,0.05323636450204631,0.05351818264721686,0.053800000792387406,0.05408181893755795,0.0543636370827285,0.05464545522789904,0.054927273373069595,0.05520909151824014,0.05549090966341069,0.05577272780858124,0.05605454595375178,0.05633636409892233,0.056618182244092874,0.056900000389263426,0.05718181853443397,0.057463636679604524,0.05774545482477507,0.058027272969945615,0.05830909111511616,0.058590909260286705,0.05887272740545725,0.0591545455506278,0.059436363695798355,0.0597181818409689,0.059999999986139446],
"y":[0.05853426559515115,0.059440563674351736,0.06032200187370409,0.06115372031335992,0.061910859113471,0.0625685583941891,0.06310626006662592,0.06353629470457775,0.06388617363449586,0.06418348974040393,0.06445583590632556,0.06472973908952033,0.06501436783809479,0.06530476876048631,0.0655955784843712,0.06588143363742574,0.06615703329160791,0.0664200263124343,0.06667222636841195,0.06691576167395072,0.0671527604434604,0.06738534950180125,0.06761511331908801,0.06784227325962079,0.0680668399324352,0.06828882394656681,0.06850823591105121,0.0687250038520804,0.0689386688767322,0.0691486588727346,0.0693544017136633,0.06955532527309406,0.0697508099350553,0.06993980410391572,0.07012102560186707,0.0702931900880531,0.07045501322161753,0.07060528707716082,0.07074436170912014,0.0708740417353447,0.07099618747526318,0.07111265924830426,0.07122530760724254,0.07133531570915766,0.07144278251409038,0.07154770765270206,0.0716500907556541,0.07174993149591964,0.07184727835045829,0.07194232259378819,0.07203528147994644,0.07212637226297008,0.07221581219689623,0.07230379960685121,0.07239042877827105,0.07247575844017036,0.

In [280]:
// Plot asset allocation with 6% risk
val assets = MutableList(n) { i -> "Asset ${i + 1}" }
val allocations = MutableList(n) { i -> 100 * xMat.get(i, 0) }

plot {
    layout.title = "Asset allocation with 6% risk"
    bars {
        x(assets) { axis.name = "" }
        y(allocations) { axis.name = "Allocation (%)" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XZBYim"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Asset allocation with 6% risk"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"name":"Allocation (%)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Asset 1","Asset 2","Asset 3","Asset 4"],
"y":[3.429043026604179E-7,29.675947879140658,66.3419470913381,3.98210468660162]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("XZBYim");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Asset 1 
 
 
 
 
 
 
 
 
 Asset 2 
 
 
 
 
 
 
 
 
 Asset 3 
 
 
 
 
 
 
 
 
 Asset 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Asset allocation with 6% risk 
 
 
 
 
 Allocation (%)